In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from modules.env import MazeVisualization
from utils.skeletonize import skeletonize_env, draw_graph_with_curved_edges, draw_scatter_plot_skeleton
import seaborn as sns
from matplotlib.patches import Circle
from matplotlib.lines import Line2D
from modules.base import SRMB
import matplotlib
from utils.topology import compute_embedding, compute_persistent_homology, compute_null_lifetimes, find_robust_holes, plot_H1_comparison_2

%matplotlib inline
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['svg.fonttype'] = 'none'
matplotlib.rcParams['ps.fonttype'] = 42
rcParams['font.sans-serif'] = ['Arial']  
rcParams['font.family'] = 'sans-serif'

sns.set_style(
    'ticks',
    rc={
        'xtick.direction': 'out', 
        'ytick.direction': 'out', 
    }
)
rcParams["figure.dpi"] = 600
# Set the font to Arial
big_font = 20
mid_font = 15
small_font = 12

indicator_font = big_font
legend_font = mid_font
title_font = mid_font
axis_font = mid_font
tick_font = small_font



subplot_scale_factor = 3

indicator_x_loc = -0.1
indicator_y_loc = 1.15

node_size = 300
edge_width = 4

junction_color = "#416788"
deadend_color = "#646464"
edge_color = "#E1DAD2"

palette = ['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ffa600']

In [2]:

num_row, num_column = 5, 3
fig, axs = plt.subplots(num_row, num_column, figsize=(num_column * 3, num_row * 3))
num_envs = 0



# A
fig.text(0.5, 1.05, 'Structure generalization', ha='center', fontsize=title_font, transform=axs[0, 1].transAxes)

env1 = MazeVisualization(5, 5)
walls1 = [
    (0, 1), (1, 2), (5, 6), (6, 7), (7, 8), (10, 11), (11, 12), (12, 13), (13, 14),
    (16, 17), (17, 18), (18, 19), (22, 23), (23, 24)
]
env1.update_map(walls=walls1)

vertex_nodes = [3, 4, 8, 9, 15, 16, 20, 21]
deadend_nodes = [0, 1, 23, 24]
edge_nodes = np.setdiff1d(np.arange(25), np.concatenate([vertex_nodes, deadend_nodes]))

env1.visualize(ax=axs[0, 0], vertex_states=[3, 4, 8, 9, 15, 16, 20, 21], deadend_states=[0, 1, 23, 24], edge_states=edge_nodes, junction_color=junction_color, deadend_color=deadend_color, edge_color=edge_color) 
axs[0, 0].text(indicator_x_loc, indicator_y_loc, 'A', transform=axs[0, 0].transAxes, fontsize=indicator_font, fontweight='bold', va='top', ha='right')


env2 = MazeVisualization(5, 5)
walls2 = [
    (0, 1), (1, 2), (5, 6), (6, 7), (7, 8), (10, 11), (11, 12), (12, 13), (13, 14),
    (16, 17), (17, 18), (8, 9), (2, 3), (3, 4)
]
blocks = [2, 7, 12, 17]
env2.update_map(blocks=blocks, walls=walls2)
env2.visualize(ax=axs[0, 1], vertex_states=[15, 16, 20, 21, 18, 19, 23, 24], deadend_states=[0, 1, 3, 4], edge_states=[5, 10, 6, 11, 8, 13, 9, 14, 22], junction_color=junction_color, deadend_color=deadend_color, edge_color=edge_color)

vertex_nodes, deadend_nodes, edge_nodes, simplified_G, image_original, skeleton, vertex_corresp, deadend_corresp, edge_corresp, _, _, _ = skeletonize_env(env2)
pos = {
    (0, 0): (0, 6),
    (0, 1): (4, 6),
    (3, 0): (2, 4),
    (3, 3): (2, 2),
    (0, 3): (0, 0),
    (0, 4): (4, 0)
}
draw_graph_with_curved_edges(simplified_G, '', rad=0.2, self_loop_rad=1, save=True, pos=pos, ax=axs[0, 2], set_margin=True, node_size=node_size, edge_width=edge_width,
)

# 범례 항목 추가
axs[0, 2].plot([], [], 'o', color=junction_color, label='Junction', markersize=10)
axs[0, 2].plot([], [], 'o', color=deadend_color, label='Dead-end', markersize=10)
axs[0, 2].plot([], [], '-', color=edge_color, label='Edge', linewidth=2)

# 범례 표시
axs[0, 2].legend(loc='center left', bbox_to_anchor=(-0.3, 0.5), frameon=False, fontsize=legend_font)

# 서브플롯 숨기기
axs[0, 2].axis('off')


# B
fig.text(0.5, 1.05, 'Reward generalization', ha='center', fontsize=title_font, transform=axs[1, 1].transAxes)

axs[1, 0].text(indicator_x_loc, indicator_y_loc, 'B', transform=axs[1, 0].transAxes, fontsize=indicator_font, fontweight='bold', va='top', ha='right')

env1.visualize(ax=axs[1, 0], subgoals=[20, 4])
env1.visualize(ax=axs[1, 1], subgoals=[16, 8])

draw_graph_with_curved_edges(
    simplified_G, '', rad=0.2, self_loop_rad=1, save=True, pos=pos, ax=axs[1, 2], set_margin=True, node_size=node_size, edge_width=edge_width,
)

# 노드 위치 정규화 (0과 1 사이)
x_min, x_max = axs[1, 2].get_xlim()
y_min, y_max = axs[1, 2].get_ylim()

normalized_pos = {}
for node, (x, y) in pos.items():
    x_norm = (x - x_min) / (x_max - x_min)
    y_norm = (y - y_min) / (y_max - y_min)
    normalized_pos[node] = (x_norm, y_norm)


circle_start = Circle(normalized_pos[(0, 0)], 0.09, edgecolor="blue", facecolor='none', transform=axs[1, 2].transAxes, linewidth=5)
circle_goal = Circle(normalized_pos[(0, 4)], 0.09, edgecolor="red", facecolor='none', transform=axs[1, 2].transAxes, linewidth=5)
circle_subgoal_1 = Circle(normalized_pos[(3, 0)], 0.09, edgecolor="orange", facecolor='none', transform=axs[1, 2].transAxes, linewidth=5)
circle_subgoal_2 = Circle(normalized_pos[(3, 3)], 0.09, edgecolor="orange", facecolor='none', transform=axs[1, 2].transAxes, linewidth=5)
axs[1, 2].add_patch(circle_start)
axs[1, 2].add_patch(circle_goal)
axs[1, 2].add_patch(circle_subgoal_1)
axs[1, 2].add_patch(circle_subgoal_2)


# Legend에 추가할 Patch 객체 생성
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='none', markeredgecolor='blue', markersize=10, label='Start', markeredgewidth=3),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='none', markeredgecolor='red', markersize=10, label='Goal', markeredgewidth=3),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='none', markeredgecolor='orange', markersize=10, label='Subgoal', markeredgewidth=3)
]

# 범례에 추가된 patch 객체 포함
legend = axs[1, 2].legend(handles=legend_elements, loc='center left', bbox_to_anchor=(-0.3, 0.5), frameon=False, fontsize=legend_font)


# C

axs[2, 0].text(indicator_x_loc, indicator_y_loc, 'C', transform=axs[2, 0].transAxes, fontsize=indicator_font, fontweight='bold', va='top', ha='right')

env3 = MazeVisualization(7, 7)
nonblocks = [0, 1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 13, 20, 27, 34, 41, 48, 43, 44, 45, 46, 47, 10, 17, 24, 31, 38]
blocks = np.setdiff1d(np.arange(49), nonblocks)
env3.update_map(blocks=blocks)
# env.scale_map(2)
env3.set_index("doubled")
env3.visualize(
    ax=axs[2, 0],
    display=True,
    puddle=False,
    trajectory=False,
    no_startgoal=True,
)
axs[2, 0].set_title("Double D maze", fontsize=title_font)
srmb = SRMB(7, 7)
srmb_bias = SRMB(7, 7)
srmb.update_map_structure(blocks=env3.blocks)
srmb_bias.update_map_structure(blocks=env3.blocks, rd_prob=0.9)
successormap_unbiased = srmb.return_successormap()
successormap_biased = srmb_bias.return_successormap()
vertex_nodes_3, deadend_nodes_3, edge_nodes_3, simplified_G_3, image_3, skeleton_3, _, _, _, _, _, _ = (
    skeletonize_env(env3)
)

node_mapping = {}
for node in simplified_G.nodes:
    node_mapping[(node)] = (11, 6) if node == (10, 6) else (node[0], 6)


# Visualize the graph
draw_graph_with_curved_edges(
    simplified_G_3, "", save=False, ax=axs[2, 1], rad=1, node_size=node_size, edge_width=edge_width,
)
axs[2, 1].set_title("Topology graph", fontsize=title_font)

draw_scatter_plot_skeleton(
    successormap_unbiased[env3.nonblocks][:, :2],
    vertex_nodes_3,
    deadend_nodes_3,
    edge_nodes_3,
    env3.nonblocks,
    save=False,
    sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=axs[3, 0],
    fig=fig,
)
axs[3, 0].set_title("Unbiased grid code", fontsize=title_font)

draw_scatter_plot_skeleton(
    successormap_biased[env3.nonblocks][:, :2],
    vertex_nodes_3,
    deadend_nodes_3,
    edge_nodes_3,
    env3.nonblocks,
    save=False,
    sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=axs[3, 1],
    fig=fig,
)
axs[3, 1].set_title("Biased grid code", fontsize=title_font)

# D

axs[2, 2].text(indicator_x_loc, indicator_y_loc, 'D', transform=axs[2, 2].transAxes, fontsize=indicator_font, fontweight='bold', va='top', ha='right')

geodist_mean_filename = 'value_geodist_spearmanr_mean.npy'
geodist_std_filename = 'value_geodist_spearmanr_std.npy'

eucliddist_mean_filename = 'value_eucliddist_spearmanr_mean.npy'
eucliddist_std_filename = 'value_eucliddist_spearmanr_std.npy'

succ_unbiased_filename_mean = "value_diffdist_spearmanr_mean_unbiased_calc_successor_map_0.99_99.npy"
succ_unbiased_filename_std = "value_diffdist_spearmanr_std_unbiased_calc_successor_map_0.99_99.npy"

diff_unbiased_filename_mean = (
    "value_diffdist_spearmanr_mean_unbiased_calc_diffusion_map_7_99.npy"
)
diff_unbiased_filename_std = (
    "value_diffdist_spearmanr_std_unbiased_calc_diffusion_map_7_99.npy"
)


base_path = "data/dist_corr"

# Load data
geodist_mean = np.load(f"{base_path}/{geodist_mean_filename}")
geodist_std = np.load(f"{base_path}/{geodist_std_filename}")
eucliddist_mean = np.load(f"{base_path}/{eucliddist_mean_filename}")
eucliddist_std = np.load(f"{base_path}/{eucliddist_std_filename}")
succ_unbiased_mean = np.load(f"{base_path}/{succ_unbiased_filename_mean}")
succ_unbiased_std = np.load(f"{base_path}/{succ_unbiased_filename_std}")
diff_unbiased_mean = np.load(f"{base_path}/{diff_unbiased_filename_mean}")
diff_unbiased_std = np.load(f"{base_path}/{diff_unbiased_filename_std}")


# Define colors
color = "#B0B0B0"

# Data for plotting
means = [succ_unbiased_mean, diff_unbiased_mean, eucliddist_mean]
cis = [succ_unbiased_std, diff_unbiased_std, eucliddist_std]
categories = ["Successor\ndistance", "Diffusion\ndistance", "Euclidean\ndistance"]
x = np.arange(len(categories))


# Plot bars with black borders
bars = axs[2, 2].bar(x, means, yerr=cis, color=color, edgecolor='black', width=0.6, error_kw={'elinewidth': 2})

# Set labels and ticks
axs[2, 2].set_ylabel("Structure encoding score\n(Spearmanr correlation)", fontsize=axis_font)
axs[2, 2].set_xticks(x)
axs[2, 2].set_xticklabels(categories, fontsize=tick_font)
axs[2, 2].set_yticks([0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
axs[2, 2].set_ylim(0.3, 1.0)
axs[2, 2].tick_params(axis='x', labelsize=tick_font, length=0)
axs[2, 2].tick_params(axis='y', labelsize=tick_font)
axs[2, 2].spines['top'].set_visible(False)
axs[2, 2].spines['right'].set_visible(False)

# E

axs[3, 2].text(indicator_x_loc, indicator_y_loc, 'E', transform=axs[3, 2].transAxes, fontsize=indicator_font, fontweight='bold', va='top', ha='right')

succ_unbiased_filename_mean = "value_diffdist_spearmanr_mean_unbiased_calc_successor_map_0.99_99.npy"
succ_unbiased_filename_std = "value_diffdist_spearmanr_std_unbiased_calc_successor_map_0.99_99.npy"
diff_biased_filename_mean = "value_diffdist_spearmanr_mean_biased_calc_successor_map_0.99_99.npy"
diff_biased_filename_std = "value_diffdist_spearmanr_std_biased_calc_successor_map_0.99_99.npy"
sr_unbiased_filename_mean = "value_sr_spearmanr_mean_unbiased_0.99.npy"
sr_unbiased_filename_std = "value_sr_spearmanr_std_unbiased_0.99.npy"
sr_biased_filename_mean = "value_sr_spearmanr_mean_biased_0.99.npy"
sr_biased_filename_std = "value_sr_spearmanr_std_biased_0.99.npy"

succ_unbiased_mean = np.load(f"{base_path}/{succ_unbiased_filename_mean}")
succ_unbiased_std = np.load(f"{base_path}/{succ_unbiased_filename_std}")
succ_biased_mean = np.load(f"{base_path}/{diff_biased_filename_mean}")
succ_biased_std = np.load(f"{base_path}/{diff_biased_filename_std}")
sr_unbiased_mean = np.load(f"{base_path}/{sr_unbiased_filename_mean}")
sr_unbiased_std = np.load(f"{base_path}/{sr_unbiased_filename_std}")
sr_biased_mean = np.load(f"{base_path}/{sr_biased_filename_mean}")
sr_biased_std = np.load(f"{base_path}/{sr_biased_filename_std}")


# Data for plotting
means = [succ_unbiased_mean, succ_biased_mean]
cis = [succ_unbiased_std, succ_biased_std]
categories = ["Unbiased\nsuccessor dist", "Biased\nsuccessor dist"]
x = np.arange(len(categories))


# Plot bars with black borders
bars = axs[3, 2].bar(x, means, yerr=cis, color=['#808080', '#C0C0C0'], edgecolor="black", width=0.6, error_kw={'elinewidth': 2})

# Set labels and ticks
axs[3, 2].set_ylabel("Structure encoding score\n(Spearmanr correlation)", fontsize=axis_font)
axs[3, 2].set_xticks(x)
axs[3, 2].set_xticklabels(categories, fontsize=tick_font)
axs[3, 2].set_yticks([0.6, 0.7, 0.8, 0.9, 1.0])
axs[3, 2].set_ylim(0.6, 1.0)
axs[3, 2].spines['top'].set_visible(False)
axs[3, 2].spines['right'].set_visible(False)
axs[3, 2].tick_params(axis='x', labelsize=tick_font, length=0)
axs[3, 2].tick_params(axis='y', labelsize=tick_font)


# 2) 저차원 임베딩 (예: 3차원)
embedding = compute_embedding(
    successormap_unbiased[env3.nonblocks],
    n_components=3,   # 임베딩 차원
    n_neighbors=3    # Isomap 이웃 개수
)

# 3) 원본 데이터에 대한 PH 계산
dgms_unbiased = compute_persistent_homology(embedding, maxdim=2)

# 4) Null 분포 생성 (셔플링 기반)
null_lf = compute_null_lifetimes(
    successormap_unbiased[env3.nonblocks],
    n_null=200,       # 반복 횟수
    n_components=3,
    n_neighbors=10,
    maxdim=2
)

# 5) Robust hole 찾기 (99.9th 퍼센타일 기준)
thresholds_unbiased, robust = find_robust_holes(dgms_unbiased, null_lf, percentile=99.9)

# 2) 저차원 임베딩 (예: 3차원)
embedding = compute_embedding(
    successormap_biased[env3.nonblocks], n_components=3, n_neighbors=3  # 임베딩 차원  # Isomap 이웃 개수
)

# 3) 원본 데이터에 대한 PH 계산
dgms_biased = compute_persistent_homology(embedding, maxdim=2)

# 4) Null 분포 생성 (셔플링 기반)
null_lf = compute_null_lifetimes(
    successormap_biased[env3.nonblocks], n_null=200, n_components=3, n_neighbors=10, maxdim=2  # 반복 횟수
)

# 5) Robust hole 찾기 (99.9th 퍼센타일 기준)
thresholds_biased, robust = find_robust_holes(dgms_biased, null_lf, percentile=99.9)

def sample_edge(p1, p2, n_points=20):
    p1 = np.array(p1)
    p2 = np.array(p2)
    return np.linspace(p1, p2, n_points)

# 그래프(ground truth) point cloud 생성
num_points = 40
theta = np.linspace(0, np.pi, num_points)
radius = 3
center_x = 3
center_y = 3

# center: 직선
center_points = np.column_stack([np.linspace(0, 6, num_points), np.full(num_points, 3)])

left_points = np.column_stack([
    center_x + radius * np.cos(theta),
    center_y - radius * np.sin(theta)
])

right_points = np.column_stack([
    center_x + radius * np.cos(theta),
    center_y + radius * np.sin(theta)
])

points_graph = np.vstack([center_points, left_points, right_points])

embedding_graph = compute_embedding(points_graph, n_components=3, n_neighbors=3)


dgms_graph = compute_persistent_homology(embedding_graph, maxdim=2)

null_lf_graph = compute_null_lifetimes(
    points_graph, n_null=200, n_components=3, n_neighbors=10, maxdim=2
)

thresholds_graph, robust_graph = find_robust_holes(dgms_graph, null_lf_graph, percentile=99.9)

plot_H1_comparison_2(
    dgms_unbiased, thresholds_unbiased[1],
    dgms_biased, thresholds_biased[1],
    ax=axs[4, 0],
    labels=("Unbiased", "Biased")
)
plt.savefig('./figure/figure_1.svg', bbox_inches='tight', transparent=True)
plt.show()
